In [3]:
import os
import sys
import json
sys.path.append('c:\\Users\\chrmol\\OneDrive - Ab Yrkeshogskolan vid Abo Akademi\\Documents\\projects\\ai-information-extraction\\pdf_information_extractor')
from src.searchable_pdf import SearchablePDF

In [5]:
pdf_path = 'data/he-specification.pdf'
json_schema_path = 'data/he-specification_schema.json'
with open(json_schema_path) as json_schema_file:
    json_schema_contents = json_schema_file.read()            
    json_schema_string = json.dumps(json.loads(json_schema_contents))

searchablePDF = SearchablePDF(pdf=pdf_path, json_schema_string=json_schema_string)

AttributeError: 'list' object has no attribute 'save'